In [2]:
from sentence_transformers import SentenceTransformer, util
import pandas as pd
import torch
import csv
import numpy as np
from sklearn.metrics import f1_score, recall_score, accuracy_score

# 检测GPU
if torch.cuda.is_available():
    device = torch.device('cuda')
    print(f'使用 GPU: {torch.cuda.get_device_name(0)}')
else:
    device = torch.device('cpu')
    print('未检测到 GPU，使用 CPU')

# 加载专为中文优化的句向量模型
model = SentenceTransformer('shibing624/text2vec-base-chinese')
model = model.to(device)

print("CUDA 是否可用:", torch.cuda.is_available())
print("当前 PyTorch 版本:", torch.__version__)
print("CUDA 版本:", torch.version.cuda if torch.cuda.is_available() else "无")

使用 GPU: NVIDIA GeForce RTX 4050 Laptop GPU
CUDA 是否可用: True
当前 PyTorch 版本: 2.6.0+cu118
CUDA 版本: 11.8


## 读取训练集并做基础统计

In [3]:
import pandas as pd
train_df = pd.read_csv('csv/test.csv', sep='\t', header=None, names=['q1', 'q2', 'label'])
print('数据集样本数:', len(train_df))
print('正负样本分布:')
print(train_df['label'].value_counts())
train_df.head(10)

数据集样本数: 4401
正负样本分布:
label
1    2237
0    2164
Name: count, dtype: int64


,q1,q2,label
0,"跪求,英雄联盟好玩还是梦三国好玩?坐等呢","亲说下,英雄联盟好玩还是梦三国好玩?十分多谢",1
1,英雄联盟统治战场一天可以得多少经验,英雄联盟打统治战场会增加隐藏分吗,0
2,那里有免费的单机游戏下载,哪里有免费的单机游戏下载,1
3,平安夜的时候送女朋友什么礼物好呢？,今年的平安夜快要到了，送女朋友什么礼物好呢？,1
4,游戏王游戏在哪下载？,游戏王游戏在哪里下载,1
5,龙飞凤舞不一般的生肖是什么？,龙飞凤舞是什么生肖？,0
6,女生不喜欢肌肉男吗？,女生都喜欢肌肉男吗？,0
7,繁体字赛，宝贝怎么写,偶的宝贝繁体字怎么写啊,0
8,这手机怎么样，好不好用啊。,充电宝怎么样才算充满,0
9,魔兽世界燃铁矿哪里多,魔兽世界-魔铁矿石哪里多,0


## - 相似度分布可视化

## 高效批量推理：大幅加速相似度计算
将所有问题对批量编码，利用向量化操作一次性计算余弦相似度，大幅提升推理速度，适合大数据量场景。

In [9]:
## 避免显存溢出：分批计算余弦相似度
#一次性计算全部对角线会导致显存溢出。可以分批处理（如每1万对），每批计算后拼接结果，显著降低显存占用。

In [4]:
# 批量编码所有问题对
q1_list = train_df['q1'].tolist()
q2_list = train_df['q2'].tolist()

# 批量编码（可指定 batch_size，适当调大可加速）
q1_emb = model.encode(q1_list, convert_to_tensor=True, batch_size=128, show_progress_bar=True)
q2_emb = model.encode(q2_list, convert_to_tensor=True, batch_size=128, show_progress_bar=True)

# 分批计算对角线相似度
batch_size = 8000
sims = []
for start in range(0, len(q1_list), batch_size):
    end = min(start + batch_size, len(q1_list))
    emb1 = q1_emb[start:end]
    emb2 = q2_emb[start:end]
    sim = util.cos_sim(emb1, emb2).diagonal().cpu().numpy()
    sims.append(sim)
sims = np.concatenate(sims)

# 设定阈值，批量判断
threshold = 0.84
train_df['pred'] = (sims > threshold).astype(int)

# 计算指标
accuracy = accuracy_score(train_df['label'], train_df['pred'])
f1 = f1_score(train_df['label'], train_df['pred'])
recall = recall_score(train_df['label'], train_df['pred'])
print(f'训练集 Accuracy: {accuracy:.4f}')
print(f'训练集 F1-score: {f1:.4f}')
print(f'训练集 Recall: {recall:.4f}')
final_score = 0.6 * f1 + 0.2 * accuracy + 0.2 * recall
print(f'加权最终得分: {final_score:.4f}')

Batches:   0%|          | 0/35 [00:00<?, ?it/s]

Batches:   0%|          | 0/35 [00:00<?, ?it/s]

训练集 Accuracy: 0.7885
训练集 F1-score: 0.8181
训练集 Recall: 0.9361
加权最终得分: 0.8358


# 错误样本分析
分析模型预测错误的前10个样本，便于人工检查模型常见误判类型和后续优化方向。

In [21]:

# 如果 'pred' 不存在，提醒用户先运行推理代码
if "pred" not in train_df.columns:
    print("请先运行模型推理代码，生成 train_df['pred'] 列！")
else:
    # 找出预测错误的样本
    wrong = train_df[train_df["pred"] != train_df["label"]]
    # 展示前10条错误样本
    print(wrong[["q1", "q2", "label", "pred"]].head(10))
    # 统计错误类型分布
    print("错误类型分布：")
    print(wrong["label"].value_counts())
    # 统计正例被误判为负例、负例被误判为正例的数量
    fp = ((wrong["label"] == 0) & (wrong["pred"] == 1)).sum()
    fn = ((wrong["label"] == 1) & (wrong["pred"] == 0)).sum()
    print(f"负例被误判为正例（假阳性）: {fp}")
    print(f"正例被误判为负例（假阴性）: {fn}")

                                q1                               q2  label  \
5                   龙飞凤舞不一般的生肖是什么？                       龙飞凤舞是什么生肖？      0   
6                       女生不喜欢肌肉男吗？                       女生都喜欢肌肉男吗？      0   
9                       魔兽世界燃铁矿哪里多                     魔兽世界-魔铁矿石哪里多      0   
11                   杭州今年这个冬天会下雪吗？                      今天杭州真的下雪了吗？      0   
12                      家电什么时候买便宜？                    什么时候买家具家电最实惠？      1   
35                      这张图出自哪个漫画？                       这张图出自什么动漫？      0   
36                     可以在网上找女朋友吗！                       我们在网上找女朋友吗      0   
47                     刚插入时女人是什么感觉                       被男人进入是什么感觉      1   
50  您好，请问您的这个新手机号是什么时间购买的呢？比如几几年几月  请问您还得这个手机号是什么购买的吗？具体到月份的时间还记得么？      1   
53                         用香皂洗脸好吗                         香皂能用来洗脸吗      0   

    pred  
5      1  
6      1  
9      1  
11     1  
12     0  
35     1  
36     1  
47     0  
50     0  
53     1  
错误类型分布：
label
0    7

In [6]:
# 筛选负例被误判为正例的样本
wrong_neg = wrong[wrong['label'] == 0]

# 计算这些样本的文本相似度
wrong_neg['similarity'] = wrong_neg.apply(
    lambda row: util.cos_sim(
        model.encode(row['q1'], convert_to_tensor=True),
        model.encode(row['q2'], convert_to_tensor=True)
    ).item(), axis=1
)

# 按相似度降序排序
wrong_neg_sorted = wrong_neg.sort_values(by='similarity', ascending=False)

# 展示相似度最高的前10个样本
print("相似度最高的负例被误判为正例样本：")
print(wrong_neg_sorted[["q1", "q2", "label", "pred", "similarity"]].head(10))

相似度最高的负例被误判为正例样本：
                 q1              q2  label  pred  similarity
910     为什么我左脸比右脸大?     为什么我右脸比左脸大?      0     1    0.992651
1957      对女生表白唱什么歌        女生表白唱什么歌      0     1    0.985824
2283  湛江到广州高铁什么时候开通  广州到湛江的高铁什么时候开通      0     1    0.983393
4021  男生说女生可爱是什么意思？  女生说男生可爱是什么意思啊？      0     1    0.980977
2818     天龙八部逍遥怎么加点     新天龙八部逍遥怎么加点      0     1    0.979994
1974       千钧一发什么意思       千钧一发均什么意思      0     1    0.977643
4210   浅绿色裤子配什么颜色鞋子  浅绿色的鞋子配什么颜色的裤子      0     1    0.976655
3382    女人喜欢男人什么部位？     男人喜欢女人什么部位？      0     1    0.975251
1825    男人喜欢戴眼镜的女人吗    女人喜欢戴眼镜的男人吗？      0     1    0.975098
3335       女人吃什么药补血         女人吃什么补血      0     1    0.974906


C:\Users\18794\AppData\Local\Temp\ipykernel_19516\3953476473.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wrong_neg['similarity'] = wrong_neg.apply(


## 替换 HanLP 依存句法分析为 LTP，并实现主谓宾结构对比辅助二次筛选
本节将使用 LTP 进行依存句法分析，抽取主谓宾三元组，并对比两句主谓宾结构，辅助判断模型误判的负例是否可二次筛选为正例。

In [ ]:
# 自动安装 ltp（如未安装）
try:
    from ltp import LTP
except ImportError:
    import sys
    !{sys.executable} -m pip install ltp
    from ltp import LTP

ltp = LTP()

def extract_svo(seg, dep):
    """
    抽取主谓宾三元组
    seg: 分词结果
    dep: 依存句法分析结果
    返回：[(主语, 谓语, 宾语), ...]
    """
    svos = []
    for i, (word, rel) in enumerate(zip(seg[0], dep[0])):
        if rel[1] == 'SBV':  # 主语
            verb = seg[0][rel[2]-1]
            subject = word
            # 查找动词的宾语
            for j, (w2, r2) in enumerate(dep[0]):
                if r2[1] == 'VOB' and r2[2]-1 == rel[2]-1:
                    obj = w2
                    svos.append((subject, verb, obj))
    return svos

def svo_match(svo_a, svo_b):
    """
    判断两个句子的主谓宾三元组是否有重合
    """
    return any(x == y for x in svo_a for y in svo_b)

# 示例：对比两句话的主谓宾结构
sent1 = "他昨天在家吃了一个大苹果"
sent2 = "昨天他在家吃苹果"

seg1, hidden1 = ltp.seg([sent1])
dep1 = ltp.dep(hidden1)
seg2, hidden2 = ltp.seg([sent2])
dep2 = ltp.dep(hidden2)

svo1 = extract_svo(seg1, dep1)
svo2 = extract_svo(seg2, dep2)
print("句1主谓宾：", svo1)
print("句2主谓宾：", svo2)
print("主谓宾结构是否一致：", svo_match(svo1, svo2))

In [ ]:
# 如果 'pred' 不存在，提醒用户先运行推理代码
if "pred" not in train_df.columns:
    print("请先运行模型推理代码，生成 train_df['pred'] 列！")
else:
    # 找出预测错误的样本
    wrong = train_df[train_df["pred"] != train_df["label"]]
    # 展示前10条错误样本
    print(wrong[["q1", "q2", "label", "pred"]].head(10))
    # 统计错误类型分布
    print("错误类型分布：")
    print(wrong["label"].value_counts())
    # 统计正例被误判为负例、负例被误判为正例的数量
    fp = ((wrong["label"] == 0) & (wrong["pred"] == 1)).sum()
    fn = ((wrong["label"] == 1) & (wrong["pred"] == 0)).sum()
    print(f"负例被误判为正例（假阳性）: {fp}")
    print(f"正例被误判为负例（假阴性）: {fn}")